# Pré-Processamento

In [1]:
from nltk.tokenize.regexp import RegexpTokenizer

trainSet = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Training set 1.txt", 'r', encoding = "ISO-8859-1")

categorias = []
perguntas = []
perguntas_finais = []

for line in trainSet:
    categorias.append(line.strip().split(':')[0])
    perguntas.append(line.strip().split(':')[1])
    
for element in perguntas:
    perguntas_finais.append(element.strip().split(" ", 1)[1])

#print(perguntas[0])
#print(perguntas_finais)

trainSet = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Training set 1.txt", 'r', encoding = "ISO-8859-1")

def preprocessamento(pergunta):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(pergunta)
    
    tokens = [word for word in tokens]
    
    return ' '.join(tokens)

trainSetTokenized = []

for pergunta in perguntas_finais:
    trainSetTokenized.append(preprocessamento(pergunta))
    
#print(trainSetTokenized[0])
    
trainSet.close()
trainSet.close()

In [2]:
from nltk.tokenize.regexp import RegexpTokenizer

testSet = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Test set.txt", 'r', encoding = "ISO-8859-1")

categoriasTest = []
perguntasTest = []
perguntas_finaisTest = []

for line in testSet:
    categoriasTest.append(line.strip().split(':')[0])
    perguntasTest.append(line.strip().split(':')[1])
    
for element in perguntasTest:
    perguntas_finaisTest.append(element.strip().split(" ", 1)[1])

#print(perguntas_finaisTest)

testSet = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Test set.txt", 'r', encoding = "ISO-8859-1")

def preprocessamentoTest(pergunta):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(pergunta)
    
    tokens = [word for word in tokens]
    
    return ' '.join(tokens)

testSetTokenized = []

for pergunta in perguntas_finaisTest:
    testSetTokenized.append(preprocessamentoTest(pergunta))
    
testSet.close()
testSet.close()

In [3]:
from collections import Counter

Counter(categorias).keys()

dict_keys(['DESC', 'ENTY', 'ABBR', 'HUM', 'NUM', 'LOC'])

In [4]:
Counter(categorias).values()

dict_values([211, 244, 18, 220, 151, 156])

## BOW

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow_model  = vectorizer.fit(trainSetTokenized)

X_bow_train = bow_model.transform(trainSetTokenized)
X_bow_test  = bow_model.transform(testSetTokenized)

print(X_bow_train.shape, X_bow_test.shape)

(1000, 2699) (500, 2699)


## Tf

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=False)
tf_model = vectorizer.fit(trainSetTokenized)

X_tf_train = tf_model.transform(trainSetTokenized)
X_tf_test  = tf_model.transform(testSetTokenized)

print(X_tf_train.shape, X_tf_test.shape)

(1000, 2699) (500, 2699)


## Tfidf

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True)
tfidf_model = vectorizer.fit(trainSetTokenized)

X_tfidf_train = tfidf_model.transform(trainSetTokenized)
X_tfidf_test  = tfidf_model.transform(testSetTokenized)

print(X_tfidf_train.shape, X_tfidf_test.shape)

(1000, 2699) (500, 2699)


# Aplicação dos Métodos

## KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
#Com o BOW

acc = -1

for i in range(2, len(categorias)):
    neigh = KNeighborsClassifier(n_neighbors = i, weights = 'distance')
    neigh.fit(X_bow_train, categorias)

    current_acc = neigh.score(X_bow_test, categoriasTest)
    
    if(current_acc > acc):
        acc = current_acc
        n = i

print("Acurácia: ", acc)
print("n: ", n)

Acurácia:  0.658
n:  2


In [22]:
#Com o Tf

acc = -1

for i in range(2, len(categorias)):
    neigh = KNeighborsClassifier(n_neighbors = i, weights = 'distance')
    neigh.fit(X_tf_train, categorias)

    current_acc = neigh.score(X_tf_test, categoriasTest)
    
    if(current_acc > acc):
        acc = current_acc
        n = i

print("Acurácia: ", acc)
print("n: ", n)

Acurácia:  0.738
n:  2


In [23]:
#Com o Tfidf

acc = -1

for i in range(2, len(categorias)):
    neigh = KNeighborsClassifier(n_neighbors = i, weights = 'distance')
    neigh.fit(X_tfidf_train, categorias)

    current_acc = neigh.score(X_tfidf_test, categoriasTest)
    
    if(current_acc > acc):
        acc = current_acc
        n = i

print("Acurácia: ", acc)
print("n: ", n)

Acurácia:  0.754
n:  12


## Naive Bayes

In [24]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

In [25]:
#Com o BOW

clf.fit(X_bow_train, categorias)

acc = clf.score(X_bow_test, categoriasTest)
print('Acurácia: ', acc)

Acurácia:  0.542


In [26]:
#Com o Tf

clf.fit(X_tf_train, categorias)

acc = clf.score(X_tf_test, categoriasTest)
print('Acurácia: ', acc)

Acurácia:  0.49


In [27]:
#Com o Tfidf

clf.fit(X_tfidf_train, categorias)

acc = clf.score(X_tfidf_test, categoriasTest)
print('Acurácia: ', acc)

Acurácia:  0.684


## MLP

In [13]:
from sklearn.neural_network import MLPClassifier

In [17]:
#Com BOW

acc = -1

for i in range(2, 101):
    mlp = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (i), random_state = 0)
    mlp.fit(X_bow_train, categorias)

    current_acc = mlp.score(X_bow_test, categoriasTest)
    
    if(current_acc > acc):
        acc = current_acc
        n = i

print('Acurácia: ', acc)
print('Número de camadas escondidas: ', n)

Acurácia:  0.788
Número de camadas escondidas:  37


In [18]:
#Com o Tf

acc= -1

for i in range(2, 101):
    mlp = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (i), random_state = 0)
    mlp.fit(X_tf_train, categorias)

    current_acc = mlp.score(X_tf_test, categoriasTest)
    
    if(current_acc > acc):
        acc = current_acc
        n = i

print('Acurácia: ', acc)
print('Número de camadas escondidas: ', n)

Acurácia:  0.792
Número de camadas escondidas:  4


In [19]:
#Com o Tfidf

acc = -1

for i in range(2, 101):
    mlp = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (i), random_state = 0)
    mlp.fit(X_tfidf_train, categorias)

    current_acc = mlp.score(X_tfidf_test, categoriasTest)
    
    if(current_acc > acc):
        acc = current_acc
        n = i

print('Acurácia: ', acc)
print('Número de camadas escondidas: ', n)

Acurácia:  0.81
Número de camadas escondidas:  64
